In [1]:
import numpy as np
import pandas as pd
import csv
import copy
from pprint import pprint
from math import *

In [2]:
dataset = pd.read_csv('Train_E.csv')
#handling missing values
dataset = dataset.fillna(dataset.mean())
#kicking date...
dataset = dataset.drop(['date'], axis = 1)
#one hot encoding for categorical columns (iso_code, continent, location)

dummy_iso_code = pd.get_dummies(dataset['iso_code'], prefix = "iso_code")
dummy_continent = pd.get_dummies(dataset['continent'], prefix = "continent")
dummy_location = pd.get_dummies(dataset['location'], prefix = "location")
#print(dummy_iso_code.head())
#print(dummy_continent.head())
#print(dummy_location.head())
dataset = pd.concat([dummy_location, dataset], axis = 1)
dataset = pd.concat([dummy_continent, dataset], axis = 1)
dataset = pd.concat([dummy_iso_code, dataset], axis = 1)
dataset = dataset.drop(['iso_code', 'continent', 'location'], axis = 1)
#dataset.head()

In [3]:
def train_test_split(s_dataset):
    k=int(len(s_dataset.index)*0.8)
    training_data = s_dataset.iloc[:k].reset_index(drop=True)
    testing_data = s_dataset.iloc[k:].reset_index(drop=True)
    return training_data,testing_data

In [4]:
#random shuffle
dataset = dataset.sample(frac=1, random_state = 0).reset_index(drop=True)
training_data,testing_data=train_test_split(dataset)
training_data.head()

,iso_code_ABW,iso_code_AFG,iso_code_AGO,iso_code_ALB,iso_code_ARE,continent_Africa,continent_Asia,continent_Europe,continent_North America,location_Afghanistan,location_Albania,location_Angola,location_Aruba,location_United Arab Emirates,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,diabetes_prevalence,life_expectancy
0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,39486.0,145.0,35.286,1467.0,5.0,1.571,1014.325,3.725,0.906,37.685,0.128,0.040,57.665208,38928341,54.422,18.6,2.581,1.337,1803.987,9.59,64.83
1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,5.0,0.0,0.143,0.0,0.0,0.000,0.506,0.000,0.014,0.000,0.000,0.000,2.780000,9890400,112.442,34.0,1.144,0.526,67293.483,17.26,77.97
2,0,0,0,0,1,0,1,0,0,0,0,0,0,1,93090.0,995.0,949.000,416.0,3.0,1.571,9412.157,100.603,95.952,42.061,0.303,0.159,52.780000,9890400,112.442,34.0,1.144,0.526,67293.483,17.26,77.97
3,0,0,1,0,0,1,0,0,0,0,0,1,0,0,5370.0,0.0,81.857,193.0,0.0,2.429,163.389,0.000,2.491,5.872,0.000,0.074,57.665208,32866268,23.890,16.8,2.405,1.362,5819.495,3.94,61.15
4,1,0,0,0,0,0,0,0,1,0,0,0,1,0,3799.0,43.0,48.429,25.0,0.0,0.286,35582.489,402.750,453.595,234.157,0.000,2.676,57.870000,106766,584.800,41.2,13.085,7.452,35973.781,11.62,76.29


In [5]:
def k_fold_validation(training_data,k):
  train_list=list()
  validation_list=list()
  l=len(training_data.index)
  fold=int(l/k)
  for i in range(k):
    validation_list.append(training_data.iloc[i*fold:(i+1)*int(l/k)].reset_index(drop=True))
    train_copy=copy.deepcopy(training_data)
    train_list.append(train_copy.drop(train_copy.index[i*fold:(i+1)*int(l/k)]).reset_index(drop=True))
  return train_list,validation_list

In [6]:
train_list,validation_list=k_fold_validation(training_data,5)
print(train_list)
print('allo allo')
print(validation_list)

[     iso_code_ABW  iso_code_AFG  ...  diabetes_prevalence  life_expectancy
0               0             0  ...                17.26            77.97
1               1             0  ...                11.62            76.29
2               0             0  ...                17.26            77.97
3               0             1  ...                 9.59            64.83
4               0             1  ...                 9.59            64.83
..            ...           ...  ...                  ...              ...
747             0             0  ...                17.26            77.97
748             1             0  ...                11.62            76.29
749             1             0  ...                11.62            76.29
750             0             1  ...                 9.59            64.83
751             1             0  ...                11.62            76.29

[752 rows x 35 columns],      iso_code_ABW  iso_code_AFG  ...  diabetes_prevalence  life_expectanc

In [7]:
def summary_util(train_data):
    result = list()
    attributes = train_data.columns
    for attribute in attributes:
      if (attribute != 'life_expectancy'):
        result.append([train_data[attribute].mean(), train_data[attribute].std()])
    return result

def summary(train_data):
    l=len(train_data.index)
    result = {}
    classes = train_data['life_expectancy'].unique()
    freq=train_data['life_expectancy'].value_counts()
    prior_probs={}
    for label in classes:
        prior_probs[label]=freq[label]/l;
        result[label] = summary_util(train_data[train_data['life_expectancy'] == label])
    return result,prior_probs

#testing abv stuff...
# print(train_list[0])
# pprint(summary(train_list[0])[0])

def gaussian(x, mean, stdev):
    if stdev==0:
      return float(x==mean)
    exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

#for i in range(5):
#  summary=summarise(train_list[i])    #need to implement
#  accuracy(summary,validation_list[i])#need to implement
  

0.0


In [8]:
def probability(row,summ_by_class):
  prob=1
  i=0
  for item in row.index:
    if(item!='life_expectancy'):
      prob*=gaussian(row[item],summ_by_class[i][0],summ_by_class[i][1])
      i+=1      
  return prob

In [9]:
def test(train_data,test_data):
  sep_class,prior_probs=summary(train_data)
  predicted_class=[]
  for i,row in test_data.iterrows():
    prob_by_class={}   
    #print(row)
    for item in sep_class:      
      prob_by_class[item]=probability(row,sep_class[item])*prior_probs[item]
    predicted_class.append(max(prob_by_class, key=prob_by_class.get))
  return predicted_class

In [10]:
def accuracy(train_data,test_data):
  predicted_classes=test(train_data,test_data)
  correct=0
  for i in range(len(predicted_classes)):
    if predicted_classes[i]==test_data['life_expectancy'][i]:
      correct+=1
  return 100*correct/len(predicted_classes)

In [11]:
for i in range(5):
  print('validation-accuracy for set'+ str(i+1)+': '+str(accuracy(train_list[i],validation_list[i])))
print('test-accuracy: '+str(accuracy(training_data,testing_data)))

validation-accuracy for set1: 100.0
validation-accuracy for set2: 100.0
validation-accuracy for set3: 100.0
validation-accuracy for set4: 100.0
validation-accuracy for set5: 100.0
test-accuracy: 100.0
